## Accessing NOAA U.S. Tabular Climate Normals with the Planetary Computer STAC API

The [NOAA U.S. Tabular Climate Normals](https://www.ncei.noaa.gov/products/land-based-station/us-climate-normals) provide information about typical climate conditions for thousands of weather station locations across the United States. Normals act both as a ruler to compare current weather and as a predictor of conditions in the near future. The official normals are calculated for a uniform 30 year period, and consist of annual/seasonal, monthly, daily, and hourly averages and statistics of temperature, precipitation, and other climatological variables for each weather station.

This Collection contains tabular weather variable data at weather station locations in GeoParquet format. Data are provided for annual/seasonal, monthly, daily, and hourly frequencies for the following time periods:

- 30-year Conventional (1991–2020)
- 30-year Legacy (1981–2010)
- 15-year Supplemental (2006–2020)

Documentation for this dataset is available at the [Planetary Computer Data Catalog](https://planetarycomputer.microsoft.com/dataset/noaa-climate-normals-tabular).


### Data Access
This notebook works with or without an API key, but you will be given more permissive access to the data with an API key. The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) sets the environment variable "PC_SDK_SUBSCRIPTION_KEY" when your server is started. The API key may be manually set via the following code:

```python
pc.settings.set_subscription_key(<YOUR API Key>)
```

The datasets hosted by the Planetary Computer are available from [Azure Blob Storage](https://docs.microsoft.com/en-us/azure/storage/blobs/). We'll use [pystac-client](https://pystac-client.readthedocs.io/) to search the Planetary Computer's [STAC API](https://planetarycomputer.microsoft.com/api/stac/v1/docs) for the subset of the data that we care about, and then we'll load the data directly from Azure Blob Storage. We'll specify a `modifier` so that we can access the data stored in the Planetary Computer's private Blob Storage Containers. See [Reading from the STAC API](https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/) and [Using tokens for data access](https://planetarycomputer.microsoft.com/docs/concepts/sas/) for more. 

In [ ]:
import planetary_computer
import pystac_client

# Open the Planetary Computer STAC API
catalog = pystac_client.Client.open(
    "https://pct-apis-staging.westeurope.cloudapp.azure.com/stac/",
    modifier=planetary_computer.sign_inplace,
)

collection = catalog.get_collection("noaa-climate-normals-tabular")
collection

In [ ]:
# Get Items
items = list(collection.get_all_items())
items

### Available Assets & Metadata

Let's display the available assets and metadata for the NOAA U.S. Tabular Climate Normals product. 

In [ ]:
import rich.table

# Assets
t_assets = rich.table.Table("Key", "Value")
for key, asset in items[0].assets.items():
    t_assets.add_row(key, asset.title)
t_assets

In [ ]:
# Metadata
t_metadata = rich.table.Table("Key", "Value")
for k, v in sorted(items[0].properties.items()):
    t_metadata.add_row(k, str(v))
t_metadata

### Loading the GeoParquet dataset
Now let's load a STAC item into a tabular format. 

In [ ]:
import pandas as pd

item = collection.get_item(id="1991_2020-monthly")
asset = item.assets["geoparquet"]

df = pd.read_parquet(
    asset.href,
    storage_options=asset.extra_fields["table:storage_options"],
    columns=["STATION", "month", "LATITUDE", "LONGITUDE", "MLY-PRCP-NORMAL"],
)

# Filter for April
df1 = df[df.month.eq(4)]
df1.head()

In [ ]:
# Descriptive statistics
stat = df1.describe()
stat.round(1)

### Displaying the tabular data

Let's display the NOAA U.S. Tabular Climate Normals.

In [ ]:
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import seaborn as sns

fig = plt.figure(figsize=(32, 32))
ax = fig.add_subplot(projection=ccrs.Mercator())
extent = [-125, -65, 24, 50.5]

lat = df1["LATITUDE"].tolist()
lon = df1["LONGITUDE"].tolist()
var = df1["MLY-PRCP-NORMAL"].tolist()

sns.scatterplot(
    x=lon,
    y=lat,
    data=df1,
    hue=var,
    palette="plasma",
    size=var,
    sizes=(50, 500),
    transform=ccrs.PlateCarree(),
    legend="auto",
    zorder=2,
    alpha=0.5,
    edgecolor="black",
)

ax.set_extent(extent)
ax.add_feature(cartopy.feature.STATES, zorder=1)
ax.add_feature(cfeature.COASTLINE, zorder=1, linewidth=1)
ax.add_feature(cfeature.OCEAN, zorder=0, linewidth=0, facecolor="#333333")
ax.add_feature(cfeature.LAND, zorder=0, linewidth=0, facecolor="#5A5A5A")
ax.set_title(
    "NOAA U.S. Tabular Climate Normals \n 1991-2020 April Precipitation Normal (inches)",
    fontweight="bold",
    fontsize="35",
)

plt.legend(fontsize=15, title="inches", title_fontsize="20", prop={"size": 25})
plt.show()